In [1]:
class UnionFindSet: 
    def __init__(self, n):
        self._parents = [i for i in range(n + 1)]
        self._ranks = [1 for i in range(n + 1)]
        
    def find(self, u):
        while u != self._parents[u]:
            self.parents[u] = self._parents[self._parents[u]]
            u = self._parents[u]
        return u 
    
    def union(self, u, v):
        pu, pv = self.find(u), self.find(v)
        if pu == pv:
            return False         
        if self._ranks[pu] < self._ranks[pv]:
            self._parents[pu] = pv
        elif self._ranks[pu] > self._ranks[pv]:
            self._parents[pv] = pu
        else:
            self._parents[pv] = pu
            self._ranks[pu] += 1
        return True 

## 399. Evaluate Division

Equantions are given in the format A/B = k, where A and B are variables represented as strings, and k is a real number (floating point number). Given some queries, return the answers. If the answer doest not exist, return -1.0

**Example:**

Given a / b = 2.0, b / c = 3.0

queries are: a/ c = ?, b/a = ?, a/e = ?, a/a = ?, x/x = ?. 

return [6.0, 0.5, -1.0, 1.0, -1.0]

According to the example above:

equations = [ ["a", "b"], ["b", "c"]],

values = [2.0, 3.0]

queries  = [ ["a", "c"], ["b", "a"], ["a", "e"], ["a", "a"], 
["x", "x"]]

In [1]:
import collections 

### Approach 1. Graph + DFS 

In [14]:
def calcEquation(equations: list, values: list, queries: list) -> list:
    
    def divide(x,y,visited):
        if x == y:
            return 1.0
        for n in g[x]:
            if n in visited: 
                continue
            visited.add(n)
            d = divide(n,y, visited)
            if d > 0: return d *g[n][x]
        return -1.0
    g = collections.defaultdict(dict)
    for (x,y), v in zip(equations, values):
        g[x][y] = v
        g[y][x] = 1.0/v
        
    ans = [divide(x, y, set()) if x in g and y in g else -1 for 
          x, y in queries]
    
    return ans 
    

In [3]:
equations = [["a", "b"], ["b", "c"]]

values = [2.0, 3.0]

g = collections.defaultdict(dict)

for (x,y), v in zip(equations, values):
    print(x,y)
    print(v)
    g[x][y] = v
    g[y][x] = 1.0/v
    
print(g)

a b
2.0
b c
3.0
defaultdict(<class 'dict'>, {'a': {'b': 2.0}, 'b': {'a': 0.5, 'c': 3.0}, 'c': {'b': 0.3333333333333333}})


In [12]:
def divide(x,y, visited):
    print('visited is ')
    print(visited)
    if x == y: 
        return 1.0
    visited.add(x)
    print('g[x] is ')
    print(g[x])
    for n in g[x]:
        print('n is ')
        print(n)
        if n in visited:
            continue
        visited.add(n)
        d = divide(n, y, visited)
        print('d is ')
        print(d)
        if d > 0: 
            return d*g[x][n]
    return -1.0 


In [13]:
x = "a"

y = "c"

visited = set()

divide(x,y,visited)

visited is 
set()
g[x] is 
{'b': 2.0}
n is 
b
visited is 
{'b', 'a'}
g[x] is 
{'a': 0.5, 'c': 3.0}
n is 
a
n is 
c
visited is 
{'c', 'b', 'a'}
d is 
1.0
d is 
3.0


6.0

In [16]:
x = "a"

y = "e"

visited = set()

divide(x,y,visited)

visited is 
set()
g[x] is 
{'b': 2.0}
n is 
b
visited is 
{'b', 'a'}
g[x] is 
{'a': 0.5, 'c': 3.0}
n is 
a
n is 
c
visited is 
{'c', 'b', 'a'}
g[x] is 
{'b': 0.3333333333333333}
n is 
b
d is 
-1.0
d is 
-1.0


-1.0

In [18]:
x in g and y in g

False

Time Complexity: O(e + q*e)

Space Complexity: O(e)

In [2]:
equations = [["a", "b"], ["b", "c"]]

values = [2.0, 3.0]

In [4]:
import collections
g = collections.defaultdict(dict)
for (x,y), v in zip(equations, values):
    g[x][y] = v
    g[y][x] = 1.0/v

In [6]:
x = "b"

for z in g[x]:
    print(z)

a
c


In [13]:
def calcEquation(equations: list, values: list, queries: list) -> list:
    g = collections.defaultdict(dict)
    
    for (x,y), v in zip(equations, values):
        g[x][y] = v
        g[y][x] = 1.0/v
        
    def divide(x, y, visited):
        if x == y:
            return 1.0
        visited.add(x)
        for z in g[x]:
            if z in visited:
                continue 
            d = divide(z, y, visited)
            if d > 0: 
                return d*g[x][z]
        return -1.0
    
    ans = []
    
    for x, y in queries:
        if x in g and y in g:
            ans.append(divide(x,y,set()))
        else:
            ans.append(-1.0)
    return ans

In [14]:
queries = [["a", "c"], ["b", "a"], ["a", "e"], ["a", "a"], 
          ["x", "x"]]

calcEquation(equations, values, queries)

[6.0, 0.5, -1.0, 1.0, -1.0]

### Approach 2: Union Find 

In [15]:
def calcEquation2(equations, values, queries):
    def find(x):
        if x != U[x][0]:
            px, pv = find(U[x][0])
            U[x] = (px, U[x][1] * pv)
        return U[x]
    
    def divide(x, y):
        rx, vx = find(x)
        ry, vy = find(y)
        if rx != ry: 
            return -1.0
        return vx/vy
    
    U = {}
    for (x,y), v in zip(equations, values):
        if x not in U and y not in U:
            U[x] = (y, v)
            U[y] = (y, 1.0)
        elif x not in U:
            U[x] = (y, v)
        elif y not in U:
            U[y] = (x, 1.0/v)
        else:
            rv, vx = find(x)
            ry, vy = find(y)
            U[rx] = (ry, v/vx * vy)
            
    ans = [divide(x, y) if x in U and y in U else -1 for x, y in queries]
    return ans     

In [16]:
calcEquation2(equations, values, queries)

[6.0, 0.5, -1, 1.0, -1]

## 742. Network Delay Time

There are N network nodes, labelled 1 to N. 

Given times, a list of travel times as directed edges tims[i] = (u, v, w), where u is the source node, v is the target node, and w is the time it takes for a signal to travel from source to target. 

Now, we send a signal from a certain node K. How long will it take for all nodes to receive the signal? If it is impossible, return -1. 

In [3]:
times = [[2,1,1], [2,3,1], [3,4,1]]

### Bellman-Ford Appraoch

In [16]:
def networkDelayTime(times: list, N: int, K:int) -> int:
    max_time = int(101*100)
    dist = [max_time] * N
    dist[K-1] = 0 # distance from K to K is 0 
    for i in range(N):
        for u,v,t in times:
            dist[v-1] = min(dist[v-1], dist[u-1] + t)
    max_dist = max(dist)
    print(dist)
    if max_dist == max_time:
        return -1
    else:
        return max_dist
            

In [17]:
networkDelayTime(times,4,2)

[1, 0, 1, 2]


2

Time Complexity: O(NE)

Spae Complexity: O(N)

N is number of nodes. E is number of edges. 

## 417. Pacific Atlantic Water Flow 

Given an $m \times n$ matrix of non-negative integers representing the height of each unit cell in a continent, the "Pacific ocean" touches the left and top edges of the matrix and the "Atlantic ocean" touches the right and bottom edges. Water can flow in four directions (up, down, left, or right) from a cell to another one with height equal or lower. 

Find the list of grid coordinates where water can flow to both the Pacific and Altantic ocean. 

**Note:**

1. The order of returned grid coordinates does not matter. 

2. Both m and n are less than 150. 

**Example:**

Given the following 5*5 matrix:

Pacific ~ ~ ~ ~ ~
     ~  1 2 2 3 (5) * 
     
     ~  3 2 3 (4)(4) * 
     
     ~  2 4 (5) 3 1  *
     
     ~. (6)(7) 1 4 5 *
     
     ~ (5) 1 1 2 4 *
     
        *. *. *. * * Altantic 
        
Return:

[[0,4], [1,3], [1,4], [2,2], [3,0], [3,1], [4,0]] (positions with parentheses in above matrix). 

In [38]:
matrix = [[1,  2, 2, 3, 5],
         [3, 2, 3, 4, 4],
         [2, 4, 5, 3, 1],
         [6, 7, 1, 4, 5],
         [5, 1, 1, 2, 4]]

In [31]:
n = len(matrix)
m = len(matrix[0])

p = [[0 for _ in range(m)] for _ in range(n)]

In [34]:
print(p)

[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]


In [36]:
for x in range(n):
    for y in range(m):
        print (p[x][y])

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


### DFS Approach 

Find all the points reachable from A and P. Find the union set of these two sets. 

In [42]:
def PacificAtlantic(matrix: list) -> list: 
    if not matrix:
        return []
    n = len(matrix)
    m = len(matrix[0])
    result = []
    
    p = [[0 for _ in range(m)] for _ in range(n)]
    a = [[0 for _ in range(m)] for _ in range(n)]
    
    def dfs(matrix, x, y, h, v):
        if (x < 0 or y < 0 or x == len(matrix) or y == len(matrix[0])):
            return 
        if v[x][y] or matrix[x][y] < h:
            return 
        v[x][y] = 1
        h = matrix[x][y]
        dfs(matrix, x-1, y, h, v)
        dfs(matrix, x+1, y, h, v)
        dfs(matrix, x, y-1, h, v)
        dfs(matrix, x, y+1, h, v)
    
    for x in range(n):
        dfs(matrix, x, 0, 0, p)
        dfs(matrix, x, m-1, 0, a)
        
    for y in range(m):
        dfs(matrix, 0, y, 0, p)
        dfs(matrix, n-1, y, 0, a)
        
    print(p)
    print(a)
        
    for x in range(n):
        for y in range(m):
            if p[x][y] and a[x][y]:
                result.append((x,y))
    return result 
    
    
    
    

In [43]:
PacificAtlantic(matrix)

[[1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 0]]
[[0, 0, 0, 0, 1], [0, 0, 0, 1, 1], [0, 0, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]


[(0, 4), (1, 3), (1, 4), (2, 2), (3, 0), (3, 1), (4, 0)]

Time Complexity: O(m + n + mn)

## 113. Path Sum II

Given a binary tree and a sum, find all root-to-leaft paths where each path's sum equals the given sum. 

In [44]:
class TreeNode:
    def __init__(self,x):
        self.val = x
        self.left = None
        self.right = None

In [46]:
def pathSum(root: TreeNode, sum_: int) -> list:
    
    ans = []
    if not root:
        return ans 
    
    def dfs(node, val, path):
        if not node:
            return 
        val -= node.val
        if not node.left and not node.right and val == 0:
            path += [node.val]
            ans.append(path)
        if val <0:
            return    
        dfs(node.left, val, path)
        dfs(node.right, val, path)
        
    dfs(root, sum_, [])
    
    

In [48]:
class Solution:
    def pathSum(self, root: TreeNode, sum: int) -> list:
        def dfs(node, path, need):
            if node and not node.left and not node.right:
                if need - node.val == 0: 
                    res.append(path+[node.val])
                    return  
            
            if not node: return
            dfs(node.left, path+[node.val], need-node.val)
            dfs(node.right, path+[node.val], need-node.val)
            

        res = []
        dfs(root, [], sum)
        
        return [] if not root else res

## 684. Redundant Connection 

In this problem, a tree is an undirected graph that is connected and has no cycles. 

The given input is a graph that started as a tree with N nodes (with distinct values 1, 2, ..., N), with one additional edge added. The added edge has two different vertices chosen from 1 to N, and was not an edge that already existed. 

The resulting graph is given as a 2D-array of edges. Each element of edges is a pair [u, v] with u < v, that represents an undirected edge connecting nodes u and v. 

Return an edge that can be removed so that the resulting graph is a tree of N nodes. If there are multiple answers, return the answer that occurs last in the given 2D-array. The answer edge [u,v] should be in the same format, with u < v. 

**Example 1:**

Input: [[1,2], [1,3], [2,3]]

Output: [2,3]

**Example 2:**

Input: [[1,2], [2,3], [3,4], [1,4], [1,5]]

Output: [1,4]



In [ ]:
def findRedundantConnection(edges: list) -> list:
    
    